In [1]:

import time
import traceback
import datetime
import requests
import urllib
import json
import os
import glob
import pathlib
import pandas as pd
import numpy as np
# import openpyxl
# os.chdir("/tf/cc/mojing")
print(os.getcwd())
pd.options.display.max_rows = 999

/home/cc/code/mojing_jupyter/20210625_奶制品提标签


# 低温酸奶测试用例

In [4]:
file_path = "./data_labeled/测试用例_错误.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")
import weightunit as unitparser
from weightunit import parse_str, UnitLevel, default_mu_merge, SingleUnit, MultiUnit

def calc_weight(rows):
    title = rows["title"]
    skuname = rows["skuname"]

    try:
        ul = UnitLevel.gen_default()
        # ul.fix_unit_level()
        mu1 = parse_str(title, unit_level=ul)
        mu2 = parse_str(skuname, unit_level=ul)
        if unitparser.DEBUG: print("mu_title=", mu1, "mu_skuname=",mu2)
        if mu1 is None and mu2 is None:
            return '空'
        elif mu2 is None:
            if mu1.have_level(1):
                return mu1.calc_weight()
            else:
                # todo 判断mu1的level是否正确
                return '空'
        elif mu1 is None:
            if mu2.have_level(1):
                return mu2.calc_weight()
            else:
                # TODO 判断mu2的level是否正确
                return '空'
                pass
        else:
            # title, skuname中都值
            if mu2.have_level(1):
                return mu2.calc_weight()
            elif mu1.have_level(1):
                # skuname有值，但没有1级单位
                # 用skuname的值，替换title中的对应单位
                # mu1  = default_mu_merge([mu1, mu2])
                mu2 = mu2.simple_formula()
                for su in mu2:
                    if isinstance(su, MultiUnit) and len(su)==1 and su.get_first_single_unit().total_flag==1 \
                        or (isinstance(su, SingleUnit) and su.total_flag==1) :
                        mu1.append(su, op='*', same_unit='verify')
                        continue
                    if len(mu2)==1 and \
                            mu1.sum_level(mu2.get_first_single_unit().level)==float(mu2.get_first_single_unit().number):
                        continue
                    if mu1 is not None and not mu1.replace_same_unit(su, True):
                        # skuname中只偶遇一个没有单位的数字舍弃
                        if len(mu2)==1 and mu2.org_units[0].unit=='':
                            continue
                        mu1.append(su, op="*")

                if unitparser.DEBUG: print("xxxxxxxxxx   ",mu1)
                return mu1.calc_weight()
            else:
                # mu1, mu2都没有level1
                return '空'
    except Exception as e:
        traceback.print_exc()
        return '空'
unitparser.DEBUG=True
if unitparser.DEBUG:
    i_row = 221
    df.iloc[i_row:i_row+1].apply(lambda row: calc_weight(row), axis=1)


In [5]:
unitparser.DEBUG = False
df["total-weight"].fillna('空', inplace=True)
df["fill_weight"] = df.apply(lambda row: calc_weight(row), axis=1)
df_miss = df[df["fill_weight"]!= df["total-weight"]]
print(len(df_miss)/len(df))
display(df_miss.head(99))

0.1875


,title,skuname,count,total-weight,fill_weight
3,光明优加纯牛奶200ml*12盒优+纯奶整箱 2果特价秒杀清仓多省包邮,【12包整箱】200克优加7月到期,200*12,212,2400.0
4,新希望纯牛奶系列至轻低脂至浓千岛湖苗条砖香蕉芒果巧克力牛奶,香蕉牛奶200ml*12 12盒,200*12,242400,2400.0
13,满2箱包邮 三元牛奶 小方白纯牛奶250ml*24 全脂学生奶,空,250*24,4800,6000.0
18,蒙牛特仑苏全脂纯牛奶250ml*12学生早餐奶整箱礼盒装 3箱起包邮,空,250*12,3000,9000.0
31,3.3g乳蛋白纯牛奶250ML*10,20支,250*20,5000,2500.0
34,纯牛奶1L*12/24盒2件包邮,NaN,1*1000*24,2400,12000.0
42,8-9月 1*16*205g安慕希常温酸奶原味,NaN,1*16*25,400,3280.0
46,简醇0蔗糖250g*1瓶+开啡尔草莓味200g*5瓶,NaN,200*1+200*5,1200,1250.0
56,君乐宝简醇无糖酸奶0蔗糖梦幻盖常温代餐非脱脂酸牛奶整箱批特价,梦幻盖2箱】0蔗糖250g*20瓶【顺丰包邮】平均52.4/箱,250*20,5000,10000.0
57,【会员专享Z】蓝堡臻风味酸奶原味莓果口味200ml*12瓶24瓶礼盒装,原味酸奶2礼盒24瓶装,200*24*2,9600,4800.0


In [6]:
import glob
# 统计每个文件下的正确率

file_list = glob.glob("./data_labeled/*.xlsx")
print()
for f in file_list:
    df = pd.read_excel(f, engine="openpyxl")
    df["total-weight"].fillna('空', inplace=True)
    df["fill_weight"] = df.apply(lambda row: calc_weight(row), axis=1)
    df_miss = df[df["fill_weight"]!= df["total-weight"]]

    print( "{}\t==\t{:}%".format(f, (1-len(df_miss)/len(df))*100))


./data_labeled/纯牛奶测试用例.xlsx	==	100.0%
./data_labeled/低温牛奶测试用例.xlsx	==	99.83333333333333%
./data_labeled/奶酪测试用例.xlsx	==	96.66666666666667%
./data_labeled/常温酸奶测试用例.xlsx	==	100.0%
./data_labeled/低温酸奶测试用例.xlsx	==	99.0%
./data_labeled/测试用例_错误.xlsx	==	81.25%
./data_labeled/奶粉测试用例.xlsx	==	99.83333333333333%
